In [1]:
import math
import time
from typing import List, Tuple
import random

def read_tsp_file(filename: str) -> List[Tuple[float, float]]:
    """TSP dosyasını okuyarak şehir koordinatlarını döndürür"""
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    
    n = int(lines[0])
    cities = []
    for line in lines[1:n+1]:
        parts = line.split()
        if len(parts) >= 2:
            x, y = float(parts[-2]), float(parts[-1])
            cities.append((x, y))
    
    return cities

def calculate_distance(city1: Tuple[float, float], city2: Tuple[float, float]) -> float:
    """İki şehir arasındaki mesafeyi hesaplar"""
    dx = city1[0] - city2[0]
    dy = city1[1] - city2[1]
    return math.sqrt(dx*dx + dy*dy)

def calculate_total_distance(route: List[int], cities: List[Tuple[float, float]]) -> float:
    """Rotanın toplam mesafesini hesaplar (başlangıç noktasına dönüş dahil)"""
    total = 0.0
    n = len(route)
    for i in range(n):
        city1 = cities[route[i]]
        city2 = cities[route[(i+1)%n]]  # Döngüsel olarak sonraki şehir
        total += calculate_distance(city1, city2)
    return total

def two_opt_swap(route: List[int], i: int, k: int) -> List[int]:
    """2-opt swap işlemi uygular"""
    new_route = route[:i]
    new_route.extend(reversed(route[i:k+1]))
    new_route.extend(route[k+1:])
    return new_route

def two_opt(cities: List[Tuple[float, float]], max_iterations: int = 1000) -> Tuple[float, List[int]]:
    """2-opt algoritması ile TSP çözümü bulur"""
    n = len(cities)
    current_route = list(range(n))
    random.shuffle(current_route)  # Rastgele başlangıç çözümü
    
    best_route = current_route.copy()
    best_distance = calculate_total_distance(best_route, cities)
    
    improved = True
    iterations = 0
    
    while improved and iterations < max_iterations:
        improved = False
        for i in range(n-1):
            for k in range(i+1, n):
                if k == i+1:  # Ardışık şehirler
                    continue
                
                new_route = two_opt_swap(current_route, i, k)
                new_distance = calculate_total_distance(new_route, cities)
                
                if new_distance < best_distance:
                    best_route = new_route
                    best_distance = new_distance
                    current_route = new_route
                    improved = True
                    break  # İyileşme bulundu, yeni tur başlat
            
            if improved:
                break
        
        iterations += 1
    
    return best_distance, best_route

def solve_tsp(filename: str) -> Tuple[float, List[int]]:
    """TSP problemini çözer"""
    print(f"{filename} dosyası işleniyor...")
    cities = read_tsp_file(filename)
    if not cities:
        raise ValueError("Şehir verisi okunamadı")
    
    print(f"{len(cities)} şehir yüklendi")
    
    print("2-opt algoritması çalışıyor...")
    start_time = time.time()
    optimal_cost, optimal_path = two_opt(cities)
    end_time = time.time()
    print(f"Çözüm bulundu, süre: {end_time - start_time:.2f} saniye")
    
    return optimal_cost, optimal_path

def save_results(filename: str, optimal_cost: float, path: List[int]):
    """Sonuçları dosyaya kaydeder"""
    with open(filename, 'w') as f:
        f.write(f"Optimal maliyet değeri: {optimal_cost}\n")
        # Rotayı kapalı hale getir (son noktadan başlangıca dön)
        closed_path = path + [path[0]]
        f.write("Optimal maliyeti sağlayan path: " + " -> ".join(map(str, closed_path)) + "\n")

if __name__ == "__main__":
    input_file = "tsp_51.txt"  # Veri dosyanızın adı
    output_file = "tsp_results.txt"
    
    try:
        optimal_cost, optimal_path = solve_tsp(input_file)
        
        print("\nSonuçlar:")
        print(f"Optimal maliyet değeri: {optimal_cost}")
        # Çıktıda rotayı kapalı göster
        print(f"Optimal yol: {' -> '.join(map(str, optimal_path))} -> {optimal_path[0]}")
        
        save_results(output_file, optimal_cost, optimal_path)
        print(f"\nSonuçlar {output_file} dosyasına kaydedildi.")
    except Exception as e:
        print(f"Hata oluştu: {str(e)}")

tsp_51.txt dosyası işleniyor...
51 şehir yüklendi
2-opt algoritması çalışıyor...
Çözüm bulundu, süre: 0.58 saniye

Sonuçlar:
Optimal maliyet değeri: 449.2483017161198
Optimal yol: 4 -> 8 -> 46 -> 3 -> 45 -> 10 -> 9 -> 28 -> 2 -> 5 -> 0 -> 33 -> 47 -> 26 -> 6 -> 36 -> 27 -> 41 -> 24 -> 34 -> 23 -> 12 -> 30 -> 11 -> 42 -> 29 -> 21 -> 37 -> 20 -> 1 -> 25 -> 31 -> 22 -> 48 -> 32 -> 17 -> 49 -> 50 -> 39 -> 43 -> 38 -> 15 -> 14 -> 44 -> 16 -> 18 -> 40 -> 19 -> 7 -> 13 -> 35 -> 4

Sonuçlar tsp_results.txt dosyasına kaydedildi.
